In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [0]:
import tensorflow as tf

In [0]:
path_to_file = 'shahname.txt'

In [0]:
text = open(path_to_file,'r').read()

In [0]:
text = text.replace("|", "")

In [49]:
print(text[:500])

به نام خداوند جان و خرد
کزین برتر اندیشه برنگذرد
خداوند نام و خداوند جای
خداوند روزی ده رهنمای
خداوند کیوان و گردان سپهر
فروزنده ماه و ناهید و مهر
ز نام و نشان و گمان برترست
نگارندهٔ بر شده پیکرست
به بینندگان آفریننده را
نبینی مرنجان دو بیننده را
نیابد بدو نیز اندیشه راه
که او برتر از نام و از جایگاه
سخن هر چه زین گوهران بگذرد
نیابد بدو راه جان و خرد
خرد گر سخن برگزیند همی
همان را گزیند که بیند همی
ستودن نداند کس او را چو هست
میان بندگی را ببایدت بست
خرد را و جان را همی سنجد اوی
در اندیشهٔ سخته 


In [0]:
vocab = sorted(set(text))

In [51]:
len(vocab)

47

In [0]:
char_to_ind = {char:ind for ind,char in enumerate(vocab)}

In [0]:
ind_to_char = np.array(vocab)

In [0]:
encoded_text = np.array([char_to_ind[c] for c in text])

In [0]:
seq_len = 24

In [0]:
total_num_seq = len(text) // (seq_len + 1)

In [57]:
total_num_seq

102185

In [0]:
char_dataset = tf.data.Dataset.from_tensor_slices(encoded_text)

In [0]:
sequences = char_dataset.batch(seq_len+1,drop_remainder=True)

In [0]:
def create_seq_target(seq):
    input_text = seq[:-1]
    target_text = seq[1:]
    return input_text,target_text

In [0]:
dataset = sequences.map(create_seq_target)

In [62]:
for input_text,target_text in dataset.take(1):
    print(input_text.numpy())
    print("".join(ind_to_char[input_text.numpy()]))
    print('\n')
    print(target_text.numpy())
    print("".join(ind_to_char[target_text.numpy()]))

[14 37  1 36 13 35  1 19 20 13 38 36 20  1 17 13 36  1 38  1 19 22 20  0]
به نام خداوند جان و خرد



[37  1 36 13 35  1 19 20 13 38 36 20  1 17 13 36  1 38  1 19 22 20  0 43]
ه نام خداوند جان و خرد
ک


In [0]:
batch_size = 128

In [0]:
buffer_size = 10000
dataset = dataset.shuffle(buffer_size).batch(batch_size,drop_remainder=True)

In [0]:
vocab_size = len(vocab)

In [0]:
embed_dim = 64

In [0]:
rnn_neurons = 1026

In [0]:
from tensorflow.keras.losses import sparse_categorical_crossentropy

In [0]:
def sparse_cat_loss(y_true,y_pred):
    return sparse_categorical_crossentropy(y_true,y_pred,from_logits=True)

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,GRU,Embedding

In [0]:
def create_model(vocab_size,embed_dim,rnn_neurons,batch_size):
  with tf.device('/gpu:0'):
    model = Sequential()
    model.add(Embedding(vocab_size,embed_dim,batch_input_shape=[batch_size,None]))
    model.add(GRU(rnn_neurons,return_sequences=True,stateful=True,recurrent_initializer='glorot_uniform'))
    model.add(Dense(vocab_size))
    model.compile(optimizer='adam',loss=sparse_cat_loss)
    return model

In [0]:
model = create_model(vocab_size,embed_dim,rnn_neurons,batch_size)

In [73]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (128, None, 64)           3008      
_________________________________________________________________
gru_2 (GRU)                  (128, None, 1026)         3361176   
_________________________________________________________________
dense_2 (Dense)              (128, None, 47)           48269     
Total params: 3,412,453
Trainable params: 3,412,453
Non-trainable params: 0
_________________________________________________________________


In [0]:
for input_example_batch,target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)

In [0]:
epochs = 30

In [76]:
!hostname

b0cadabcd5da


In [77]:
with tf.device('/gpu:0'):
  model.fit(dataset,epochs=epochs)

Epoch 1/30
798/798 [==============================] - 45s 56ms/step - loss: 2.0902
Epoch 2/30
798/798 [==============================] - 45s 57ms/step - loss: 1.6494
Epoch 3/30
798/798 [==============================] - 45s 57ms/step - loss: 1.5566
Epoch 4/30
798/798 [==============================] - 46s 57ms/step - loss: 1.5085
Epoch 5/30
798/798 [==============================] - 46s 57ms/step - loss: 1.4761
Epoch 6/30
798/798 [==============================] - 46s 57ms/step - loss: 1.4502
Epoch 7/30
798/798 [==============================] - 46s 57ms/step - loss: 1.4299
Epoch 8/30
798/798 [==============================] - 46s 57ms/step - loss: 1.4125
Epoch 9/30
798/798 [==============================] - 45s 57ms/step - loss: 1.3980
Epoch 10/30
798/798 [==============================] - 46s 57ms/step - loss: 1.3853
Epoch 11/30
798/798 [==============================] - 46s 57ms/step - loss: 1.3746
Epoch 12/30
798/798 [==============================] - 46s 57ms/step - loss: 1.3652
E

In [0]:
model.save('shahname.h5')

In [0]:
model = create_model(vocab_size,embed_dim,rnn_neurons,batch_size=1)

In [0]:
model.load_weights('shahname.h5')
model.build(input_shape=tf.TensorShape([1,None]))

In [0]:
def generate_text(model,start_seed,gen_size=500,temp=1.0):

  num_generate = gen_size
  input_eval = [char_to_ind[s] for s in start_seed]
  input_eval = tf.expand_dims(input_eval,0)
  text_generated = []

  temperature = temp

  model.reset_states()

  for i in range(num_generate):
    predictions = model(input_eval)
    predictions = tf.squeeze(predictions,0)

    predictions = predictions / temperature

    predicted_id = tf.random.categorical(predictions,num_samples=1)[-1,0].numpy()

    input_eval = tf.expand_dims([predicted_id],0)
    
    text_generated.append(ind_to_char[predicted_id])

  return (start_seed + ''.join(text_generated))



In [85]:
print(generate_text(model,'ز گفتار تا عود و عود و عبیر',gen_size=1000))

ز گفتار تا عود و عود و عبیر
چهارم که تابوت بد کش گنهک عاج
همه خوب چهر
کزین خاکشان فله‌های همی‌بگذرد
دل هوا روشن ومارد به سر
نکرد آن کشیدند صحرا اندرین
همه پیش برهان هلاکه کرد
زن کودکان مادر گفت کای رنج کار
به گوش سپه را سرافراز تو راست منذر سخن
یکی گرزهٔ رادرون کرد کشتی چوشید
پیاده برفتی کند ار خسپر نامبردار گرد
همی شد ز بهرام هنگه هزار
بزرگست و شادیش کردند ما گنج تو
ببرجشنگهایی همه پیش شاه ارد این سخن
بدو گفت شاها که اسکندر آیی به باراز شما شست هرگز دروغ
حل‌آوری سال نیک
به پرورد تا هم‌زن
همه خاک را خویش تست
و را پوست او رونهزار
به گیتیتری
به از راه دیوار کیست
به ز چشم بنه برنهاد
بدر و فر یزدگرد
چنان گفت پس چندی بخسرو نگردد به پیش
چنان چون بود درخور مهربان تیرباران اوی
نی ز تخت مهی را سزاست
بگفت و به شیر ژیان برگذشت
ز اکو پیش
همه پر هر به پای نشاید نهفت
مهان سوی خاک رویین فراز و نشیب
وگرنه نیوشم نبود
بدان تا رقعه و پاک بیزارمش
ز ایرانیان برفرازد همی
ن خرد
ز دینار و بر ما گذشتی سخن
تو رابا سر تاجدار سپاه
چو آمد به دل دانش و را پای کنداوران
همی بفت خواهم کنون نیست اندرکشتم آید نخست او زر

In [0]:
# pd.DataFrame(model.history.history).plot()